In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config("spark.sql.warehouse.dir", f"/user/itv010698/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
loans_repay_df = spark.read \
.format("csv") \
.option("inferSchema", "true") \
.option("header", "true") \
.load("lending_club_project/raw_data/loan_repayments_csv")

In [3]:
loans_repay_df

loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d
118184277,4500.0,309.83,0.0,4809.8270961624,3136.06,Oct-2018,null
117289602,1851.99,466.72,0.0,2318.71,128.94,Mar-2019,Apr-2019
118216186,4593.79,1285.61,0.0,5879.4,326.97,Mar-2019,Apr-2019
117584790,20000.0,1638.12,0.0,21638.120791947,16960.53,May-2018,null
118208834,7053.74,1396.74,0.0,8450.48,469.84,Mar-2019,Apr-2019
117566588,4888.37,3811.19,0.0,8699.56,484.19,Mar-2019,Apr-2019
118221134,2261.1,2017.6,0.0,4278.7,238.17,Mar-2019,Apr-2019
118183021,2688.72,345.32,0.0,3034.04,168.65,Mar-2019,Apr-2019
118209005,7567.78,1346.25,0.0,8914.03,495.58,Mar-2019,Apr-2019
118214952,3291.77,651.81,0.0,3943.58,219.26,Mar-2019,Apr-2019


In [6]:
loans_repay_df.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- total_rec_prncp: string (nullable = true)
 |-- total_rec_int: string (nullable = true)
 |-- total_rec_late_fee: string (nullable = true)
 |-- total_pymnt: string (nullable = true)
 |-- last_pymnt_amnt: string (nullable = true)
 |-- last_pymnt_d: string (nullable = true)
 |-- next_pymnt_d: string (nullable = true)



In [4]:
schema = "loan_id string, total_principal_received float, total_interest_received float, total_late_fee_received float, total_payment_received float, last_payment_amount float, last_payment_date string, next_payment_date string"

In [5]:
loans_repay_df = spark.read \
.format("csv") \
.schema(schema) \
.option("header", "true") \
.load("lending_club_project/raw_data/loan_repayments_csv")

In [6]:
loans_repay_df.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- total_principal_received: float (nullable = true)
 |-- total_interest_received: float (nullable = true)
 |-- total_late_fee_received: float (nullable = true)
 |-- total_payment_received: float (nullable = true)
 |-- last_payment_amount: float (nullable = true)
 |-- last_payment_date: string (nullable = true)
 |-- next_payment_date: string (nullable = true)



### Add ingest_date column

In [7]:
from pyspark.sql.functions import *

In [8]:
loans_repay_ingest_date = loans_repay_df.withColumn("ingest_date", current_timestamp())

In [9]:
loans_repay_ingest_date

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date
118184277,4500.0,309.83,0.0,4809.827,3136.06,Oct-2018,null,2024-03-26 11:08:...
117289602,1851.99,466.72,0.0,2318.71,128.94,Mar-2019,Apr-2019,2024-03-26 11:08:...
118216186,4593.79,1285.61,0.0,5879.4,326.97,Mar-2019,Apr-2019,2024-03-26 11:08:...
117584790,20000.0,1638.12,0.0,21638.121,16960.53,May-2018,null,2024-03-26 11:08:...
118208834,7053.74,1396.74,0.0,8450.48,469.84,Mar-2019,Apr-2019,2024-03-26 11:08:...
117566588,4888.37,3811.19,0.0,8699.56,484.19,Mar-2019,Apr-2019,2024-03-26 11:08:...
118221134,2261.1,2017.6,0.0,4278.7,238.17,Mar-2019,Apr-2019,2024-03-26 11:08:...
118183021,2688.72,345.32,0.0,3034.04,168.65,Mar-2019,Apr-2019,2024-03-26 11:08:...
118209005,7567.78,1346.25,0.0,8914.03,495.58,Mar-2019,Apr-2019,2024-03-26 11:08:...
118214952,3291.77,651.81,0.0,3943.58,219.26,Mar-2019,Apr-2019,2024-03-26 11:08:...


In [10]:
loans_repay_ingest_date.createOrReplaceTempView("loans_repay")

### Drop the rows which have nulls in any of these columns
["total_principal_received", "total_interest_received", "total_late_fee_received", "toal_payment_amount", "last_payment_amount"]

In [11]:
spark.sql("SELECT COUNT(*) FROM loans_repay WHERE total_principal_received	 is null")

count(1)
69


In [12]:
loans_repay_ingest_date.count()

2260701

In [13]:
columns_to_check = ["total_principal_received", "total_interest_received", "total_late_fee_received", "total_payment_received", "last_payment_amount"]

In [14]:
loans_repay_nulls_dropped = loans_repay_ingest_date.na.drop(how='any', subset = columns_to_check)

In [15]:
loans_repay_nulls_dropped.count()

2260498

In [16]:
loans_repay_nulls_dropped.createOrReplaceTempView("loans_repay")

### The records where total_payment_received = 0 and total_principal_received != 0  are incorrect. Need to fix them
### total_payment_received = total_principal_received + total_interest_received + total_late_fee_received

In [17]:
spark.sql("SELECT * FROM loans_repay WHERE total_payment_received = 0.0 and total_principal_received != 0.0")

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date
485818,14640.096,13388.84,13000.0,0.0,0.0,0.0,Mar-2013,2024-03-26 11:09:...
485471,29620.818,29134.64,25000.0,0.0,0.0,0.0,Mar-2013,2024-03-26 11:09:...
482256,8735.611,7479.87,8000.0,0.0,0.0,0.0,Feb-2011,2024-03-26 11:09:...
478160,410.0,407.36,0.0,0.0,143.1,410.0,null,2024-03-26 11:09:...
476557,28865.18,24164.67,5692.31,0.0,6972.59,19916.78,Dec-2010,2024-03-26 11:09:...
472516,25951.482,24731.76,25000.0,0.0,0.0,0.0,May-2010,2024-03-26 11:09:...
472197,12048.13,12018.01,10000.0,0.0,0.0,0.0,Jan-2013,2024-03-26 11:09:...
467364,29216.791,29066.19,24250.0,0.0,0.0,0.0,Dec-2012,2024-03-26 11:09:...
399499,26557.729,26336.41,24000.0,0.0,0.0,0.0,Dec-2010,2024-03-26 11:09:...
451482,7587.5513,7587.55,7000.0,0.0,0.0,0.0,Jan-2011,2024-03-26 11:09:...


In [18]:
loans_repay_clean1 = loans_repay_nulls_dropped.withColumn("total_payment_received", 
                                     when(
                                         (col("total_principal_received") != 0.0) &
                                         (col("total_payment_received") == 0.0),
                                         col("total_principal_received") + col("total_interest_received") + ("total_late_fee_received")
                                     ).otherwise(col("total_payment_received"))   
                                    )

In [19]:
loans_repay_clean1.filter("total_payment_received = 0.0 and total_principal_received != 0")

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date


### Fix the records where total_payment_received = 0.0

In [20]:
loans_repay_clean1.count()

2260498

In [21]:
loans_repay_clean2 = loans_repay_clean1.filter("total_payment_received != 0.0")

In [22]:
loans_repay_clean2.filter("total_payment_received = 0.0")

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date


In [23]:
loans_repay_clean2.count()

2259503

### Fix the last_payment_date column

In [24]:
loans_repay_clean2.select("last_payment_date").distinct()

last_payment_date
Sep-2017
Oct-2016
2590.8
Jul-2009
May-2015
Aug-2014
Apr-2013
Jan-2012
May-2010
Sep-2008


In [25]:
loans_repay_clean3 = loans_repay_clean2.filter("last_payment_date!='3572.93'")

### Fixing the last_payment_date = 0.0 and next_payment_date = 0.0 . replace them with null

In [26]:
loans_repay_clean3.filter("last_payment_date = 0.0").count()

14

In [27]:
loans_repay_clean3.filter("next_payment_date = 0.0").count()

22

In [28]:
loans_repay_ldate_fixed = loans_repay_clean3.withColumn( "last_payment_date",
                              when(col("last_payment_date") == 0.0,None)
                              .otherwise(col("last_payment_date")) 
                             )

In [29]:
loans_repay_nextdate_fixed = loans_repay_ldate_fixed.withColumn( "next_payment_date",
                              when(col("next_payment_date") == 0.0,None)
                              .otherwise(col("next_payment_date")) 
                             )

In [30]:
loans_repay_nextdate_fixed.filter("last_payment_date = 0.0").count()

0

In [31]:
loans_repay_nextdate_fixed.filter("next_payment_date = 0.0").count()

0

In [34]:
loans_repay_nextdate_fixed.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- total_principal_received: float (nullable = true)
 |-- total_interest_received: float (nullable = true)
 |-- total_late_fee_received: float (nullable = true)
 |-- total_payment_received: double (nullable = true)
 |-- last_payment_amount: float (nullable = true)
 |-- last_payment_date: string (nullable = true)
 |-- next_payment_date: string (nullable = true)
 |-- ingest_date: timestamp (nullable = false)



### write it back to the cleaned folder

In [32]:
loans_repay_nextdate_fixed.write \
.format("csv") \
.option("header", "true") \
.mode("overwrite") \
.option("path", "lending_club_project/cleaned_data/loan_repayments_cleaned_csv") \
.save()

In [33]:
loans_repay_nextdate_fixed.write \
.format("parquet") \
.mode("overwrite") \
.option("path", "lending_club_project/cleaned_data/loan_repayments_cleaned_parquet") \
.save()